In [ ]:
!pip install gymnasium

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 953.9/953.9 kB 6.7 MB/s eta 0:00:00


In [ ]:
!pip install swig

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 13.6 MB/s eta 0:00:00


In [ ]:
!pip install gymnasium[box2d]

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 374.4/374.4 kB 7.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for box2d-py: filename=box2d_py-2.3.5-cp310-cp310-linux_x86_64.whl size=2376135 sha256=d6ac461f134015c010f362ee8bcaf6e35b1c4f9e08878da5c4f14a1ebc8c6b1e
  Stored in directory: /root/.cache/pip/wheels/db/8f/6a/eaaadf056fba10a98d986f6dce954e6201ba3126926fc5ad9e
Successfully built box2d-py


In [ ]:
!pip install stable-baselines3[extra]

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 182.3/182.3 kB 5.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 434.7/434.7 kB 13.3 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 19.2 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl (121.6 MB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl (56.5 MB)
  Using cached n

In [ ]:
from dataclasses import dataclass, asdict, field
from typing import Dict

import gymnasium as gym
import numpy as np

from stable_baselines3 import PPO
from stable_baselines3.common.env_util import make_vec_env
from stable_baselines3.common.evaluation import evaluate_policy

import matplotlib.pyplot as plt

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [ ]:
@dataclass
class Params:
  normalize_advantage: bool = True
  policy: str = "MlpPolicy"
  batch_size: int = 256
  n_steps: int = 8
  gamma: float = 0.9999
  learning_rate: float = float(7.77e-05)
  ent_coef: float = 0.00429
  clip_range: float = 0.1
  n_epochs: int = 100
  gae_lambda: float = 0.9
  max_grad_norm: int = 5
  vf_coef: float = 0.19
  use_sde: bool = True
  policy_kwargs: Dict = field(default_factory=lambda: dict(log_std_init=-3.29, ortho_init=False))

In [ ]:
# Parallel environments
# "MountainCarContinuous-v0"
def evaluate(params: Params, vec_env, batch: int, episodes: int, total_steps: int, milestone: int):
  merged_dict = asdict(params) | {"env": vec_env}
  print(merged_dict)
  model = PPO(**merged_dict)
  steps = 0
  rewards = []
  rewards_per_batch = []
  while steps < total_steps:
    print("steps", steps)
    print("learning...")
    model.learn(total_timesteps=milestone)
    vec_env.reset()
    for episode in range(1, episodes + 1):
        print("ep", episode)
        state = vec_env.reset()
        done = False
        score = 0
        while not np.all(done):
            action, _ = model.predict(state)
            state, reward, done, info = vec_env.step(action)
            score += reward
        score = np.mean(score)
        rewards_per_batch.append(score)
        if episode % batch == 0:
          score = np.mean(rewards_per_batch)
          rewards_per_batch.clear()
          rewards.append(score)
          print('Episode:{} Score:{}'.format(episode, score))
    steps += milestone

  vec_env.close()
  return rewards

In [ ]:
import sys

sys.version

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


'3.10.12 (main, Nov 20 2023, 15:14:05) [GCC 11.4.0]'

In [ ]:
def visualise(rewards: list, batch: int):
  plt.figure(figsize=(10, 5))  # Set the figure size
  x, y = zip(*[(reward, index * batch) for index, reward in enumerate(rewards)])
  plt.plot(list(y), list(x)) # 'o' creates a scatter plot with dots

  plt.xlabel('Episode')
  plt.ylabel('Reward')

  plt.title('Rewards per Episode - Mountain cart continuous PPO')
  plt.savefig('rewards_plot_ppo.png')
  plt.show()

In [ ]:
def evaluate_suite(params_list: list, vec_env, batch: int, episodes: int, total_steps: int, milestone: int):
  for params in params_list:
    results = evaluate(params, vec_env, batch, episodes, total_steps, milestone)
    visualise(results, batch)

In [ ]:
def evaluate_inbuilt(params, vec_env):
  model = PPO(
      policy=params.policy,
      env=vec_env,
      learning_rate=params.learning_rate,
      n_steps=params.n_steps,
      batch_size=params.batch_size,
      gamma=params.gamma,
      gae_lambda=params.gae_lambda,
      normalize_advantage=params.normalize_advantage,
      use_sde=params.use_sde,
      sde_sample_freq=params.sde_sample_freq
  )

  mean_reward, std_reward = evaluate_policy(model, vec_env, n_eval_episodes=100, warn=False)

  print(f"mean_reward: {mean_reward:.2f} +/- {std_reward:.2f}")

In [ ]:
def evaluate_suite_inbuilt(params_list: list, vec_env):
  for params in params_list:
    evaluate_inbuilt(params, vec_env)

In [ ]:
params = Params()
vec_env = make_vec_env("MountainCarContinuous-v0", n_envs = 2)

evaluate_suite([params], vec_env, 10, 20, 10_000, 10_000)

{'normalize_advantage': True, 'policy': 'MlpPolicy', 'batch_size': 256, 'n_steps': 8, 'gamma': 0.9999, 'learning_rate': 7.77e-05, 'ent_coef': 0.00429, 'clip_range': 0.1, 'n_epochs': 100, 'gae_lambda': 0.9, 'max_grad_norm': 5, 'vf_coef': 0.19, 'use_sde': True, 'policy_kwargs': {'log_std_init': -3.29, 'ortho_init': False}, 'env': <stable_baselines3.common.vec_env.dummy_vec_env.DummyVecEnv object at 0x7d969ab3c5b0>}


/usr/local/lib/python3.10/dist-packages/stable_baselines3/ppo/ppo.py:155: UserWarning: You have specified a mini-batch size of 256, but because the `RolloutBuffer` is of size `n_steps * n_envs = 16`, after every 0 untruncated mini-batches, there will be a truncated mini-batch of size 16
We recommend using a `batch_size` that is a factor of `n_steps * n_envs`.
Info: (n_steps=8 and n_envs=2)
  warnings.warn(


steps 0
learning...


In [ ]:
evaluate_inbuilt(params, vec_env)